<a href="https://colab.research.google.com/github/ashutoshbaghel/cnn-sentence-classification/blob/master/CNN_for_Sectence_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [0]:
import numpy as np
import re
import pickle
from collections import defaultdict
from datetime import datetime

In [74]:
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.nn.utils as utils
import pickle as pk
from torch.utils.data import DataLoader, Dataset 
import time
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
print(torch.cuda.get_device_name())

cuda
Tesla T4


In [0]:
def append_to_file(my_list, fname="output.txt"):
    with open(fname, "w") as f:
        if(type(my_list)==list):
            for item in my_list:
                f.write("%s\n" % item)
        elif (type(my_list)==str):
            f.write("%s\n" % my_list)
        else:
            print("[ERROR] Only writes lists and texts to file.")

# Get files in working directory

All files are also downloadable from [Google Drive](https://drive.google.com/drive/folders/1G_AZqP5zOVRGaLe2MqE2OJVQ6xUkdhpL?usp=sharing)

In [3]:
from google.colab import drive
drive.mount('/gdrive')
# %cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!cp -a "/gdrive/My Drive/data/." .

# Preprocessing

In [0]:
def clean_str(string):
    """
    String cleaning and tokenization for dataset
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

In [0]:
val_data = np.genfromtxt("topicclass_valid.txt", delimiter="|||", dtype="str")
train_data = np.genfromtxt("topicclass_train.txt", delimiter="|||", dtype="str")
test_data = np.genfromtxt("topicclass_test.txt", delimiter="|||", dtype="str")
assert len(train_data)==253909
assert len(val_data)==643
assert len(test_data)==697

In [0]:
train_label = train_data[:,0]
train_sent = [clean_str(sent) for sent in train_data[:,1]]
val_label = val_data[:,0]
val_sent = [clean_str(sent) for sent in val_data[:,1]]
test_label = test_data[:,0]
test_sent = [clean_str(sent) for sent in test_data[:,1]]

In [0]:
def get_label_idx_map(labels):
    """
    takes list of labels from the train dataset
    return dicts which map: label->index and index->label
    """
    labels = set(labels)
    i = 0
    label_to_idx_map = {}
    idx_to_label_map = {}
    for label in labels:
        label_to_idx_map[label.strip()] = i
        idx_to_label_map[i] = label.strip()
        i += 1
    return label_to_idx_map, idx_to_label_map

In [0]:
label_idx_map, idx_label_map = get_label_idx_map(train_label)

In [12]:
# Create vocab from train set
vocab = defaultdict(float)
for sent in train_sent:
    for word in sent.split():
        vocab[word] += 1
print(len(vocab))

112760


In [0]:
# Load word2vec for words in vocab:
def load_word2vec(fname, vocab):
    word2vec = {}
    with open(fname, 'r') as f:
        header = f.readline()
        vocab_size, layer1_size = map(int, header.split())
        for i in range(vocab_size):
            line = f.readline().split()
            word, vec = line[0], np.asarray(line[1:], dtype=np.float32)
            if word in vocab:
                word2vec[word] = vec
    return word2vec

In [14]:
word2vec = load_word2vec("./GoogleNews-vectors-negative300.txt", vocab)
len(vocab) - len(word2vec)

59918

In [0]:
# TODO
# Can make this tinier, common for all unknown words 
def add_unknown_words(w2v, vocab):
    i = 0 
    for word in vocab:
        if word not in w2v:
            i +=1 
            w2v[word] = np.random.uniform(-0.25,0.25,300) 
            #change this to other logic after seeing performace
    print("Total unknown words= ", i)

In [16]:
add_unknown_words(word2vec, vocab)

Total unknown words=  59918


In [17]:
len(word2vec) == len(vocab)

True

In [0]:
def get_W(word_vecs, k=300):
    """
    Get word matrix. W[i] is the vector for word indexed by i
    """
    vocab_size = len(word_vecs)
    word_idx_map = dict()
    W = np.zeros(shape=(vocab_size+1, k), dtype=np.float32)
    W[0] = np.zeros(k, dtype=np.float32)
    i = 1
    for word in word_vecs:
        W[i] = word_vecs[word]
        word_idx_map[word] = i
        i += 1
    return W, word_idx_map

In [0]:
W, word_idx_map = get_W(word2vec)

In [0]:
pickle.dump([W, word_idx_map, label_idx_map, vocab], 
            open('w-wordmap-labelmap-vocab.pickle', 'wb'))

# Prepare Train/Val/Test

In [0]:
def get_idx_from_sent(sent, word_idx_map):
    """
    Transforms sentence into a list of indices. 
    """
    x = []
    words = sent.split()
    for word in words:
        if word in word_idx_map:
            x.append(word_idx_map[word])
    return x

In [23]:
train = [get_idx_from_sent(sent,word_idx_map) for sent in train_sent]
train_label = [label_idx_map[label.strip()] for label in train_label]
assert len(train) == len (train_label)
print(len(train))

253909


In [24]:
val = [get_idx_from_sent(sent,word_idx_map) for sent in val_sent]
val_label = [label_idx_map[label.strip()] for label in val_label]
assert len(val) == len (val_label)
print(len(val))

643


In [25]:
test = [get_idx_from_sent(sent,word_idx_map) for sent in test_sent]
test_label = np.zeros(len(test_sent))
assert len(test) == len (test_label)
print(len(test))

697


In [0]:
def save_model(model, qualifier="final"):
    torch.save(model.state_dict(), f"{model.__class__.__name__}-{qualifier}-{datetime.now().timestamp()}.pt")

# Dataset and Dataloaders

In [0]:
class SentenceDataset(Dataset):
    def __init__(self, sent, labels, W):
        self.sentences = sent
        self.labels = labels
        self.W = W
        self.word_idx_map = word_idx_map

    def __len__(self):
        return len(self.sentences)
        
    def __getitem__(self, index):
        sent = self.sentences[index]
        vec = np.zeros((len(sent), 300))
        for i,word_idx in enumerate(sent):
            vec[i] = self.W[word_idx]
        label = self.labels[index]
        return torch.tensor(vec), torch.tensor(label)

In [0]:
def collate_train(batch_data):
    '''
    '''
    X, Y = zip(*batch_data)
    X = pad_sequence(X, batch_first=True)
    return X, torch.tensor(Y)
    

In [0]:
train_dataset = SentenceDataset(train, train_label, W)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=8, pin_memory=True, collate_fn=collate_train)

In [0]:
val_dataset = SentenceDataset(val, val_label, W)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True, num_workers=8, pin_memory=True, collate_fn=collate_train)

In [0]:
test_dataset = SentenceDataset(test, test_label, W)

In [0]:
class CNN(nn.Module):
    def __init__(self, output_size, in_channels, out_channels, kernel_size, stride, padding, dropout_prob, embed_size=300): #, vocab_size, embed_size, weights):
        super(CNN, self).__init__()
        
        """       
        """
        self.output_size = output_size
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        self.conv1 = nn.Conv2d(in_channels, out_channels, (kernel_size[0], embed_size), stride, padding)
        self.conv2 = nn.Conv2d(in_channels, out_channels, (kernel_size[1], embed_size), stride, padding)
        self.conv3 = nn.Conv2d(in_channels, out_channels, (kernel_size[2], embed_size), stride, padding)
        self.dropout = nn.Dropout(dropout_prob)
        self.label = nn.Linear(len(kernel_size)*out_channels, output_size)
    
    def convolution(self, input, conv_layer):
        out = conv_layer(input)# out.size() = (batch_size, out_channels, dim, 1)
        out = F.relu(out.squeeze(3))# out.size() = (batch_size, out_channels, dim1)
        max_output = F.max_pool1d(out, out.size()[2]).squeeze(2)# max_output.size() = (batch_size, out_channels)
        return max_output
    
    def forward(self, sentences):
        
        """
        Parameters
        ----------
        sentences: sentences of shape = (batch_size, num_sequences)
        
        Returns
        -------
        logits.size() = (batch_size, output_size)
        """
        
        input = sentences.float() # input.size() = (batch_size, num_seq, embed_size)
        input = input.unsqueeze(1)      # input.size() = (batch_size, 1, num_seq, embed_size)
        max_output1 = self.convolution(input, self.conv1)
        max_output2 = self.convolution(input, self.conv2)
        max_output3 = self.convolution(input, self.conv3)
        
        out = torch.cat((max_output1, max_output2, max_output3), 1)  # out.size() = (batch_size, num_kernels*out_channels)
        fc_in = self.dropout(out)  # fc_in.size() = (batch_size, num_kernels*out_channels)
        logits = self.label(fc_in)
        return logits

In [0]:
output_size = 16
input_channels = 1 # Corresponding to each sentence
output_channels = 100
kernel_size = [3,4,5]
stride = 1
padding = (2,0) # pad word-dimension, leave embedding size as is. 
dropout_prob = 0.5

loss_fn = F.cross_entropy
num_epoch = 10 

mymodel = CNN(output_size, input_channels, output_channels, kernel_size, 
              stride, padding, dropout_prob)

In [0]:
 #TODO: Can add gradient clipping here. 
 def train_model(model, train_iter, epoch):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.cuda()
    optim = torch.optim.Adam(model.parameters())

    model.train()

    since = time.time()
    for idx, batch in enumerate(train_iter):
        text = batch[0]
        target = batch[1]
        target = torch.autograd.Variable(target).long()
        if torch.cuda.is_available():
            text = text.cuda()
            target = target.cuda()
        optim.zero_grad()
        prediction = model(text)
        loss = loss_fn(prediction, target)
        prediction = torch.max(prediction, 1)[1]
        num_corrects = (prediction.data == target.data).sum()
        acc = 100.0 * num_corrects/len(batch[0])
        loss.backward()
        optim.step()

        
        if idx % 500 == 0:
            print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.2f}, Training Accuracy: {acc.item(): .2f}%')
        
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
    end_time = time.time()    
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter), (end_time - since)

def eval_model(model, val_iter, write_to_file=False, fname="output.txt"):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            text = batch[0]
            target = batch[1]
            target = torch.autograd.Variable(target).long()
            if torch.cuda.is_available():
                text = text.cuda() 
                target = target.cuda()
            prediction = model(text)
            loss = loss_fn(prediction, target)
            prediction = torch.max(prediction, 1)[1]
            num_corrects = (prediction.data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch[0])
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()
            
            # write results to a file
            if write_to_file:
                append_to_file([idx_label_map[int(idx)] for idx in prediction], 
                               fname)

    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)

In [78]:
for epoch in range(num_epoch):
    train_loss, train_acc, time_elapsed = train_model(mymodel, train_loader, epoch)
    val_loss, val_acc = eval_model(mymodel, val_loader)
    
    print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.2f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:2f}, Val. Acc: {val_acc:.2f}%, , Time: {time_elapsed:.2f} sec')

Epoch: 1, Idx: 1, Training Loss: 2.78, Training Accuracy:  5.47%
Epoch: 1, Idx: 501, Training Loss: 1.02, Training Accuracy:  69.53%
Epoch: 1, Idx: 1001, Training Loss: 0.91, Training Accuracy:  77.34%
Epoch: 1, Idx: 1501, Training Loss: 1.00, Training Accuracy:  68.75%
Epoch: 1, Train Loss: 1.01, Train Acc: 69.37%, Val. Loss: 0.640697, Val. Acc: 81.55%, , Time: 57.21
Epoch: 2, Idx: 1, Training Loss: 0.96, Training Accuracy:  69.53%
Epoch: 2, Idx: 501, Training Loss: 0.79, Training Accuracy:  78.12%
Epoch: 2, Idx: 1001, Training Loss: 0.87, Training Accuracy:  73.44%
Epoch: 2, Idx: 1501, Training Loss: 0.78, Training Accuracy:  81.25%
Epoch: 2, Train Loss: 0.84, Train Acc: 73.96%, Val. Loss: 0.605385, Val. Acc: 81.25%, , Time: 56.80
Epoch: 3, Idx: 1, Training Loss: 0.78, Training Accuracy:  75.00%
Epoch: 3, Idx: 501, Training Loss: 0.80, Training Accuracy:  74.22%
Epoch: 3, Idx: 1001, Training Loss: 0.86, Training Accuracy:  71.09%
Epoch: 3, Idx: 1501, Training Loss: 0.68, Training Acc

In [0]:
save_model(mymodel)

In [86]:
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=8, pin_memory=True, collate_fn=collate_train)
eval_model(mymodel, val_loader, write_to_file=True, fname="dev_results.txt")

(0.6062538693796605, 81.95956454121307)

In [85]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=8, pin_memory=True, collate_fn=collate_train)
eval_model(mymodel, test_loader, write_to_file=True, fname="test_results.txt")

(8.164417405552639, 3.012912482065997)